# Exploratory Analysis

In [5]:
import pandas as pd
from nba_api.stats.static import teams
from datetime import date
from nba_api.stats.endpoints import teamgamelog, teamyearbyyearstats, scoreboardv3
from nba_api.live.nba.endpoints import scoreboard

## Get API NBA data

### Get static team data
`get_teams` returns a list of 30 dictionaries, each an NBA team. <br>
For players you could use: `players.get_players()`

In [4]:
nba_teams = teams.get_teams()
print(f"Number of teams fetched: {len(nba_teams)}")
nba_teams[:3]

Number of teams fetched: 30


[{'id': 1610612737,
  'full_name': 'Atlanta Hawks',
  'abbreviation': 'ATL',
  'nickname': 'Hawks',
  'city': 'Atlanta',
  'state': 'Georgia',
  'year_founded': 1949},
 {'id': 1610612738,
  'full_name': 'Boston Celtics',
  'abbreviation': 'BOS',
  'nickname': 'Celtics',
  'city': 'Boston',
  'state': 'Massachusetts',
  'year_founded': 1946},
 {'id': 1610612739,
  'full_name': 'Cleveland Cavaliers',
  'abbreviation': 'CLE',
  'nickname': 'Cavaliers',
  'city': 'Cleveland',
  'state': 'Ohio',
  'year_founded': 1970}]

### Explore endpoints
`teamyearbyyearstats` -> Overall teams stats year over year. <br>
`teamgamelog` -> All game stats for a particular team. <br>
`scoreboardv3` -> A give date's game scores.<br>
`scoreboard`-> Live todaýs scores

In [13]:
cav_stats = teamyearbyyearstats.TeamYearByYearStats(team_id=1610612739)
cav_stats.df = cav_stats.get_data_frames()[0]
cav_stats.df.head(3)

,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,...,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS,PTS_RANK
0,1610612739,Cleveland,Cavaliers,1970-71,82,15,67,0.183,8,4,...,0,0,0,2065,2114,0,0,0,8373,17
1,1610612739,Cleveland,Cavaliers,1971-72,82,23,59,0.280,7,4,...,0,0,0,2060,1936,0,0,0,8674,16
2,1610612739,Cleveland,Cavaliers,1972-73,82,32,50,0.390,6,4,...,0,0,0,2106,1941,0,0,0,8418,17


In [14]:
cavs_log = teamgamelog.TeamGameLog(team_id=1610612739,)
cavs_log_df = cavs_log.get_data_frames()[0]
cavs_log_df.head(3)

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612739,0022500802,"FEB 20, 2026",CLE @ CHA,W,36,21,0.632,240,41,...,0.929,11,28,39,24,10,3,12,21,118
1,1610612739,0022500793,"FEB 19, 2026",CLE vs. BKN,W,35,21,0.625,240,44,...,0.600,8,48,56,27,7,7,13,13,112
2,1610612739,0022500776,"FEB 11, 2026",CLE vs. WAS,W,34,21,0.618,240,45,...,0.865,16,35,51,30,9,8,21,15,138


In [8]:
today_date = date.today().strftime("%Y-%m-%d")
scoreboard = scoreboardv3.ScoreboardV3(
    league_id="00",
    game_date=today_date
)

game_header = scoreboard.game_header.get_data_frame()
line_score = scoreboard.line_score.get_data_frame()

line_score.head(2)

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,wins,losses,score,seed,inBonus,timeoutsRemaining
0,0022500817,1610612760,Oklahoma City,Thunder,OKC,thunder,43,14,86,0,None,3
1,0022500817,1610612739,Cleveland,Cavaliers,CLE,cavaliers,36,21,83,0,None,3


In [16]:
game_header[game_header['gameId'] == '0022500817']

,gameId,gameCode,gameStatus,gameStatusText,period,gameClock,gameTimeUTC,gameEt,regulationPeriods,seriesGameNumber,gameLabel,gameSubLabel,seriesText,ifNecessary,seriesConference,poRoundDesc,gameSubtype,isNeutral
0,0022500817,20260222/CLEOKC,2,3rd Qtr,3,PT02M42.00S,2026-02-22T18:00:00Z,2026-02-22T13:00:00Z,4,,,,,False,,,,False


In [6]:
today_date = date.today().strftime("%Y-%m-%d")
board = scoreboard.ScoreBoard()
games_dict = board.get_dict()
games_list = games_dict["scoreboard"]["games"]

rows = []
for g in games_list:
    rows.append({
        "game_id": g["gameId"],
        "status": g["gameStatusText"],
        "period": g["period"],
        "clock": g["gameClock"],

        "away_tricode": g["awayTeam"]["teamTricode"],
        "away_city": g["awayTeam"]["teamCity"],
        "away_name": g["awayTeam"]["teamName"],
        "away_score": g["awayTeam"]["score"],
        "away_wins": g["awayTeam"]["wins"],
        "away_losses": g["awayTeam"]["losses"],

        "home_tricode": g["homeTeam"]["teamTricode"],
        "home_city": g["homeTeam"]["teamCity"],
        "home_name": g["homeTeam"]["teamName"],
        "home_score": g["homeTeam"]["score"],
        "home_wins": g["homeTeam"]["wins"],
        "home_losses": g["homeTeam"]["losses"],

        "utc_time": g["gameTimeUTC"],
    })

scores_df = pd.DataFrame(rows)
preview_df = scores_df[["game_id","status","away_tricode","away_score","home_tricode","home_score","status","period","clock","utc_time"]]
game_count = len(preview_df)
print(f"{game_count} NBA games on {today_date}:")
preview_df.head(3)

11 NBA games on 2026-02-22:


,game_id,status,away_tricode,away_score,home_tricode,home_score,status,period,clock,utc_time
0,0022500817,Q3 :11.2,CLE,86,OKC,89,Q3 :11.2,3,PT00M11.20S,2026-02-22T18:00:00Z
1,0022500818,3:30 pm ET,BKN,0,ATL,0,3:30 pm ET,0,,2026-02-22T20:30:00Z
2,0022500819,3:30 pm ET,TOR,0,MIL,0,3:30 pm ET,0,,2026-02-22T20:30:00Z
